In [73]:
import numpy as np
import pandas as pd
from pandas import Series
import matplotlib as plt
%matplotlib inline
from scipy.stats import mode


In [2]:
#Import datasets and add an additional column 'Source' to split the datasets later 
train = pd.read_csv('F:/Data_Science/Datasets/BigMart3_Train.csv')
train['source'] = 'train'
test = pd.read_csv('F:/Data_Science/Datasets/BigMart3_Test.csv')
test['source'] = 'test'

In [3]:
#Combine the datasets
df =pd.concat([train, test], ignore_index=True)
print(train.shape, test.shape, df.shape)
df.head()

(8523, 13) (5681, 12) (14204, 13)


,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type,source
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,Medium,Supermarket Type1,train
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,Medium,Supermarket Type2,train
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,Medium,Supermarket Type1,train
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.000000,19.20,1998,OUT010,Tier 3,NaN,Grocery Store,train
4,Low Fat,NCD19,53.8614,994.7052,Household,0.000000,8.93,1987,OUT013,Tier 3,High,Supermarket Type1,train


In [5]:
#Information about Numerical columns
df.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000
mean,141.004977,2181.288914,0.065953,12.792854,1997.830681
std,62.086938,1706.499616,0.051459,4.652502,8.371664
min,31.290000,33.290000,0.000000,4.555000,1985.000000
25%,94.012000,834.247400,0.027036,8.710000,1987.000000
50%,142.247000,1794.331000,0.054021,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


In [6]:
#Categorical variable data
cat = df.dtypes[df.dtypes == 'object'].index
df[cat].describe()


,Item_Fat_Content,Item_Identifier,Item_Type,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type,source
count,14204,14204,14204,14204,14204,10188,14204,14204
unique,5,1559,16,10,3,3,4,2
top,Low Fat,FDA02,Fruits and Vegetables,OUT027,Tier 3,Medium,Supermarket Type1,train
freq,8485,10,2013,1559,5583,4655,9294,8523


In [4]:
#df.groupby('Outlet_Type').Outlet_Size.transform(lambda x: x.fillna(x.mode()[0]))

#Replace Item_Fat_Content with appropriate values - 1
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace(['LF', 'low fat'], 'Low Fat')

#Replace Item_Fat_Content with appropriate values - 2
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace('reg', 'Regular')

df['Item_Fat_Content'].value_counts()

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64

In [5]:
#Imputing missing values of Item_Weight grouped by Item_Identifier

print (" Before" , df['Item_Weight'].isnull().sum())
df['Item_Weight'].fillna(df.groupby('Item_Identifier')['Item_Weight'].transform("mean"), inplace = True)
print (" After" , df['Item_Weight'].isnull().sum())

 Before 2439
 After 0


In [63]:
df1 = df

In [6]:
#Imputuing missing values of Outlet_Size with mode of Outlet_Size grouped by Outlet_Type
print('Before', df['Outlet_Size'].isnull().sum())
df['Outlet_Size'] = df.groupby('Outlet_Type').Outlet_Size.transform(lambda x: x.fillna(x.mode()[0]))
print('After', df['Outlet_Size'].isnull().sum())

Before 4016
After 0


In [14]:
#finding mode of column grouped by another
df.groupby('Outlet_Type')['Outlet_Size'].agg(lambda x: x.value_counts().index[0])

Outlet_Type
Grocery Store         Small
Supermarket Type1     Small
Supermarket Type2    Medium
Supermarket Type3    Medium
Name: Outlet_Size, dtype: object

In [7]:
#Check to see if combining Outlet_Type makes sense. We can do this by checking mean sales per Outlet Type
df.groupby('Outlet_Type')['Item_Outlet_Sales'].mean().round(2)


Outlet_Type
Grocery Store         339.83
Supermarket Type1    2316.18
Supermarket Type2    1995.50
Supermarket Type3    3694.04
Name: Item_Outlet_Sales, dtype: float64

In [8]:
#Need to fix the visibility values which were zero. Every item should have a visibility
#We will fix it by mean visibility of the product
df[(df['Item_Visibility'] == 0)].count()

Item_Fat_Content             879
Item_Identifier              879
Item_MRP                     879
Item_Outlet_Sales            526
Item_Type                    879
Item_Visibility              879
Item_Weight                  879
Outlet_Establishment_Year    879
Outlet_Identifier            879
Outlet_Location_Type         879
Outlet_Size                  879
Outlet_Type                  879
source                       879
dtype: int64

In [9]:
#Replacing Item_Visibility zero values with NaN
print(df['Item_Visibility'].isnull().sum())
df['Item_Visibility'] = df['Item_Visibility'].replace(0.0, np.nan)
print(df['Item_Visibility'].isnull().sum())

0
879


In [10]:
#Imputing the missing values of Item_Visibility with mean of each Item_Identifier
df['Item_Visibility'].fillna(df.groupby('Item_Identifier')['Item_Visibility'].transform("mean"), inplace = True)
print(df['Item_Visibility'].isnull().sum())

0


In [11]:
#df = df.drop('Item_Visbility', axis = 1)
df.columns

Index(['Item_Fat_Content', 'Item_Identifier', 'Item_MRP', 'Item_Outlet_Sales',
       'Item_Type', 'Item_Visibility', 'Item_Weight',
       'Outlet_Establishment_Year', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Size', 'Outlet_Type', 'source'],
      dtype='object')

In [12]:
df['Item_Visibility'].describe()

count    14204.000000
mean         0.070458
std          0.050086
min          0.003575
25%          0.031381
50%          0.058064
75%          0.098042
max          0.328391
Name: Item_Visibility, dtype: float64

In [13]:
#Try to reduce the Item Types by creating a new variable combining Item Type and Item Identifier
df['Item_Category'] = df['Item_Identifier'].str.slice(0,2)

In [14]:
df['Item_Category'] = df['Item_Category'].map({'FD':'Food', 'DR': 'Drinks', 'NC':'Non-Consumable'})

In [15]:
df['Item_Category'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Category, dtype: int64

In [16]:
df.groupby('Item_Category')['Item_Fat_Content'].count()

Item_Category
Drinks             1317
Food              10201
Non-Consumable     2686
Name: Item_Fat_Content, dtype: int64

In [17]:
df.loc[df['Item_Category']=="Non-Consumable",'Item_Fat_Content'] = 'None'

In [18]:
df['Item_Fat_Content'].value_counts()

Low Fat    6499
Regular    5019
None       2686
Name: Item_Fat_Content, dtype: int64

In [19]:
#Integer conversions (Label Encoder)
from sklearn.preprocessing import LabelEncoder
lc = LabelEncoder()

In [20]:
df['Outlet_No'] = lc.fit_transform(df['Outlet_Identifier'])
df['Outlet_No'].value_counts()

5    1559
1    1553
6    1550
9    1550
8    1550
7    1548
3    1546
2    1543
0     925
4     880
Name: Outlet_No, dtype: int64

In [21]:
#Convert few object type fields to integers using Label Encoder
int_con = ['Item_Fat_Content','Item_Type','Outlet_Size','Outlet_Location_Type','Outlet_Type', 'Item_Category']
for i in int_con:
    df[i] = lc.fit_transform(df[i])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 15 columns):
Item_Fat_Content             14204 non-null int64
Item_Identifier              14204 non-null object
Item_MRP                     14204 non-null float64
Item_Outlet_Sales            8523 non-null float64
Item_Type                    14204 non-null int64
Item_Visibility              14204 non-null float64
Item_Weight                  14204 non-null float64
Outlet_Establishment_Year    14204 non-null int64
Outlet_Identifier            14204 non-null object
Outlet_Location_Type         14204 non-null int64
Outlet_Size                  14204 non-null int64
Outlet_Type                  14204 non-null int64
source                       14204 non-null object
Item_Category                14204 non-null int64
Outlet_No                    14204 non-null int64
dtypes: float64(4), int64(8), object(3)
memory usage: 1.6+ MB


In [29]:
df['Item_Category'].value_counts()

1    10201
2     2686
0     1317
Name: Item_Category, dtype: int64

In [22]:
#One hot encoding
df = pd.get_dummies(df, columns=['Item_Fat_Content','Item_Type','Outlet_Size','Outlet_Location_Type','Outlet_Type', 'Item_Category','Outlet_No'])
df.dtypes

Item_Identifier               object
Item_MRP                     float64
Item_Outlet_Sales            float64
Item_Visibility              float64
Item_Weight                  float64
Outlet_Establishment_Year      int64
Outlet_Identifier             object
source                        object
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Item_Fat_Content_2             uint8
Item_Type_0                    uint8
Item_Type_1                    uint8
Item_Type_2                    uint8
Item_Type_3                    uint8
Item_Type_4                    uint8
Item_Type_5                    uint8
Item_Type_6                    uint8
Item_Type_7                    uint8
Item_Type_8                    uint8
Item_Type_9                    uint8
Item_Type_10                   uint8
Item_Type_11                   uint8
Item_Type_12                   uint8
Item_Type_13                   uint8
Item_Type_14                   uint8
Item_Type_15                   uint8
O

In [23]:
#save the combined file  #df.to_excel('C:/Hari Docs/Dataset/Bigmart_cleaned.xlsx')

#Converting establishment year to years of operation of store
df['Outlet_Ops_Years'] = 2013 - df['Outlet_Establishment_Year']
df['Outlet_Ops_Years'].describe()


count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Ops_Years, dtype: float64

In [24]:
#Drop Outlet_Establishment_Years
df.drop(['Outlet_Establishment_Year'], axis = 1, inplace=True)

In [37]:
df.columns

Index(['Item_Identifier', 'Item_MRP', 'Item_Outlet_Sales', 'Item_Visibility',
       'Item_Weight', 'Outlet_Identifier', 'source', 'Item_Fat_Content_0',
       'Item_Fat_Content_1', 'Item_Fat_Content_2', 'Item_Type_0',
       'Item_Type_1', 'Item_Type_2', 'Item_Type_3', 'Item_Type_4',
       'Item_Type_5', 'Item_Type_6', 'Item_Type_7', 'Item_Type_8',
       'Item_Type_9', 'Item_Type_10', 'Item_Type_11', 'Item_Type_12',
       'Item_Type_13', 'Item_Type_14', 'Item_Type_15', 'Outlet_Size_0',
       'Outlet_Size_1', 'Outlet_Size_2', 'Outlet_Location_Type_0',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2', 'Outlet_Type_0',
       'Outlet_Type_1', 'Outlet_Type_2', 'Outlet_Type_3', 'Item_Category_0',
       'Item_Category_1', 'Item_Category_2', 'Outlet_No_0', 'Outlet_No_1',
       'Outlet_No_2', 'Outlet_No_3', 'Outlet_No_4', 'Outlet_No_5',
       'Outlet_No_6', 'Outlet_No_7', 'Outlet_No_8', 'Outlet_No_9',
       'Outlet_Ops_Years'],
      dtype='object')

In [26]:
#Dividing back into test and train dataset
train = df.loc[df['source'] == 'train']
test = df.loc[df['source'] == 'test']

In [27]:
print(train.shape, test.shape)

(8523, 50) (5681, 50)


In [28]:
test.drop(['source'], axis = 1, inplace=True)
train.drop(['source'], axis = 1, inplace=True)

F:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
F:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [97]:
train.to_csv('F:/Data_Science/Datasets/Bigmart_train_modified.csv', index=False)
test.to_csv('F:/Data_Science/Datasets/Bigmart_test_modified.csv', index=False)

In [29]:
#Complete script below is from AnalyticsVidhya and needs to be worked on by myself
#Linear Regression
from sklearn.linear_model import LinearRegression

In [30]:
#Defining Target and ID Columns
target = ['Item_Outlet_Sales']
IDcol = ['Item_Identifier','Outlet_Identifier']

predictors = [x for x in train.columns if x not in ['Item_Outlet_Sales','Item_Identifier','Outlet_Identifier']]
predictors

['Item_MRP',
 'Item_Visibility',
 'Item_Weight',
 'Item_Fat_Content_0',
 'Item_Fat_Content_1',
 'Item_Fat_Content_2',
 'Item_Type_0',
 'Item_Type_1',
 'Item_Type_2',
 'Item_Type_3',
 'Item_Type_4',
 'Item_Type_5',
 'Item_Type_6',
 'Item_Type_7',
 'Item_Type_8',
 'Item_Type_9',
 'Item_Type_10',
 'Item_Type_11',
 'Item_Type_12',
 'Item_Type_13',
 'Item_Type_14',
 'Item_Type_15',
 'Outlet_Size_0',
 'Outlet_Size_1',
 'Outlet_Size_2',
 'Outlet_Location_Type_0',
 'Outlet_Location_Type_1',
 'Outlet_Location_Type_2',
 'Outlet_Type_0',
 'Outlet_Type_1',
 'Outlet_Type_2',
 'Outlet_Type_3',
 'Item_Category_0',
 'Item_Category_1',
 'Item_Category_2',
 'Outlet_No_0',
 'Outlet_No_1',
 'Outlet_No_2',
 'Outlet_No_3',
 'Outlet_No_4',
 'Outlet_No_5',
 'Outlet_No_6',
 'Outlet_No_7',
 'Outlet_No_8',
 'Outlet_No_9',
 'Outlet_Ops_Years']

In [31]:
lin_reg = LinearRegression(normalize=True)
lin_reg.fit(train[predictors], train[target])

print(lin_reg.score(train[predictors], train[target]))


0.563720595274


In [33]:
from sklearn import cross_validation, metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from math import sqrt
#cv_score = cross_validation.cross_val_score(lin_reg, train[predictors], train[target], cv = 10, scoring = 'mean_squared_error')
#cv_score = np.sqrt(np.abs(cv_score))

#print("RMSE: %.4g" %np.sqrt(metrics.mean_squared_error(train[target].values, X_test)))

F:\Programs\Anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [59]:
prediction =  lin_reg.predict(train[predictors])
rmse = np.sqrt(metrics.mean_squared_error(train[target].values, prediction))
rmse

1127.1016134596632

In [60]:
test[target] = lin_reg.predict(test[predictors])

F:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
F:\Programs\Anaconda\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [74]:
coef1 = pd.Series(lin_reg.coef_, predictors).sort_values()
coef1
#plt.pyplot.bar(predictors, lin_reg.coef_)
#plt.pyplot.show()

Exception: Data must be 1-dimensional